In [2]:
# Import context
import sys
import os
sys.path.insert(0, os.path.abspath('..\\matlib'))
sys.path.append('..')  # Add parent directory (root)

from vec import Vec
# from mat import Mat

# Check Python version (critical for solver.py)
import imp
sys.path
print(imp.get_magic())
print(imp.get_magic()==b'\xee\x0c\r\n')  # should be true

b'\xee\x0c\r\n'
True


In [4]:
# nested comprehension for list-of-row-list representation
# of 3x4 matrix all of whose elements are zero
[[0 for _ in range(4)] for _ in range(3)]

[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]

In [3]:
class Mat:
    def __init__(self, labels, function):
        self.D = labels
        self.f = function

# Identity matrix
def identity(D): return Mat((D,D), {(k,k):1 for k in D})

"""
matrix = [ [1, 2, 3], [10, 20, 30] ]

row-labels -> matrix.D[0]: 'a', 'b'
col-labels -> matrix.D[1]: '@', '#', '?'
"""

matrix = Mat( ({'a','b'}, {'@', '#', '?'}), {('a','@'):1, ('a','#'):2, ('a','?'):3, ('b','@'):10, ('b','#'):20, ('b','?'):30} )

# row to column dictionary conversion
def mat2coldict(A):
    return {c: Vec(A.D[0], {r:A.f[r,c] for r in A.D[0]}) for c in A.D[1]}

mat2coldict(matrix)

{'#': Vec({'b', 'a'},{'b': 20, 'a': 2}),
 '?': Vec({'b', 'a'},{'b': 30, 'a': 3}),
 '@': Vec({'b', 'a'},{'b': 10, 'a': 1})}

In [4]:
!python -m doctest mat.py
# !python -m doctest mat_sparsity.py

In [7]:
# Solving Lights Out
from GF2 import zero, one

n = 2
D = {(i,j) for i in range(n) for j in range(n)}

# Button vectors
b00 = Vec(D, {(0,0):one, (0,1):one, (1,0):one})
b01 = Vec(D, {(0,0):one, (0,1):one, (1,1):one})
b10 = Vec(D, {(0,0):one, (1,0):one, (1,1):one})
b11 = Vec(D, {(0,1):one, (1,0):one, (1,1):one})

# Initial position (lights in)
b = Vec(D, {(0,0):one, (1,0):one})
print(b)

from matutil import coldict2mat
A = coldict2mat({(0,0):b00, (0,1):b01, (1,0):b10, (1,1):b11})
print(A)

from solver import solve
x = solve(A, b)
print(x)
A*x == b


 (0, 0) (0, 1) (1, 0) (1, 1)
----------------------------
    one      0    one      0

            (0, 0) (0, 1) (1, 0) (1, 1)
          -----------------------------
 (0, 0)  |     one    one    one      0
 (0, 1)  |     one    one      0    one
 (1, 0)  |     one      0    one    one
 (1, 1)  |       0    one    one    one


 (0, 0) (0, 1) (1, 0) (1, 1)
----------------------------
      0    one      0    one


True

In [14]:
from matutil import listlist2mat
A = listlist2mat([[1,3],[5,7]])
b = Vec({0,1}, {0:1, 1:1})
u = solve(A, b)
res = b - A*u  # if small => success
print(res)
print(res*res)


         0         1
--------------------
 -4.44E-16 -8.88E-16
9.860761315262648e-31


In [15]:
# Solving resource consumption (espionage) problem
from mat import Mat
col_labels={'metal','concrete','plastic','water','electricity'}
row_labels={'gnome','hoop','slinky','putty','shooter'}
M = Mat((row_labels,col_labels),
        {('gnome','concrete'):1.3,
         ('gnome','plastic'):.2,
         ('gnome','water'):.8,
         ('gnome','electricity'):.4})
#etc.

# Observed resource utilization
b = Vec(col_labels, {'electricity':1409.5, 'water':1485.0, 
                     'concrete':1300.0, 'metal':226.25,
                     'plastic':677.0})
u = solve(M.transpose(), b)
print(u)
print(u*M)  # check that solution makes sense
res = b-u*M
print(res*res)


    gnome hoop putty shooter slinky
-----------------------------------
 1.41E+03    0     0       0      0

 concrete electricity metal plastic    water
--------------------------------------------
 1.84E+03         566     0     283 1.13E+03
1333583.5733695652


In [17]:
# For some matrix-vector equations, there is no solution
# For these, the res(idual) will not be so small
from vecutil import list2vec
A = listlist2mat([[1,2], [4,5], [-6,1]])
b = list2vec([1,1,1])
u = solve(A, b) 
    # Computes the solution with res as small as possible
res = b-A*u
print(res*res)

0.24287856071964012


In [18]:
# Some matrix-vector equations are ill-conditioned, which
# can prevent an algorithm using floats from getting
# even approx. solutions, even if solution exists!
A = listlist2mat([[1e20,1],[1,0]])
b = list2vec([1,1])
u = solve(A, b)
print(u)
res = b - A*u
print(res*res)


     0 1
--------
 1E-20 0
1.0
